The goal is to replicate the following blog code:

https://aws.amazon.com/fr/blogs/machine-learning/generate-synthetic-data-for-evaluating-rag-systems-using-amazon-bedrock/

In [74]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

In [75]:
load_dotenv("../.env")

True

In [66]:
prompt_template = """
    Here is some context:
    <context>
    {}
    </context>

    Your task is to generate 5 questions in FRENCH that can be answered using the provided context and return a JSON object of a list of questions, following these rules:

    <rules>
    1. The question should make sense to humans even when read without the given context.
    2. The question should be fully answered from the given context.
    3. The question should be framed from a part of context that contains important information. It can also be from tables, code, etc.
    4. The answer to the question should not contain any links.
    5. The question should be of moderate difficulty.
    6. The question must be reasonable and must be understood and responded by humans.
    7. Do not use phrases like 'provided context', etc. in the question.
    8. Avoid framing questions using the word "and" that can be decomposed into more than one question.
    9. The question should not contain more than 10 words, make use of abbreviations wherever possible.
    </rules>

    To generate the question, first identify the most important or relevant part of the context. Then frame a question around that part that satisfies all the rules above."""

In [67]:
context = """
Ces premiers ouvrages donnèrent naissance à des compilations, à des
abrégés plus ou moins médiocres qui n'apprenoient rien de nouveau. Une
dispute qui s'éleva quelques années après, entre deux savants[3], sur
nos anciennes églises, sans éclaircir beaucoup la question qu'ils
traitoient, répandit quelques nouvelles lumières sur les antiquités de
Paris. Pendant ce temps, Henri Sauval, avocat au parlement,
travailloit à nous donner des connoissances plus exactes et plus
étendues sur un sujet aussi important. Il recueillit, dans les dépôts
publics et dans les archives particulières, une quantité prodigieuse
de documents et de titres sur l'état ancien et moderne de la ville de
Paris, les lut, les dépouilla avec une patience infatigable; mais
n'eut ni le temps ni peut-être le talent de les mettre en ordre, de
les comparer, de les vérifier. Il en est résulté que son immense
recueil n'est qu'un amas informe de matériaux confondus ensemble, et
dont il est impossible d'user sans y apporter les plus grandes
précautions. Il est plein de répétitions, de détails fatigants, de
trivialités, inexact dans les faits, peu judicieux dans les
réflexions; et ses erreurs sur une foule de matières, principalement
sur l'appréciation des monuments, sont telles, qu'elles seroient
insupportables aujourd'hui aux personnes même les moins éclairées.
"""

In [68]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt_template.format(context),
        }
    ],
    model="gpt-4o-mini",
    response_format={"type": "json_object"},
)

In [69]:
questions_json_object = json.loads(chat_completion.choices[0].message.content)
print(questions_json_object)

{'questions': ['Qui a travaillé sur les antiquités de Paris?', 'Quels problèmes rencontraient les travaux de Sauval?', 'Quel type de documents a recueilli Henri Sauval?', 'Quel est le défaut majeur du recueil de Sauval?', 'Comment étaient les erreurs de Sauval sur les monuments?']}


In [ ]:
generation_template = """
    It is your task to generate an answer to the following questions <questions>{}</questions> only based on the <context>{}</context></task>
    The output should be only the answer generated from the context.
    <rules>
    1. Only use the given context as a source for generating the answer.
    2. Be as precise as possible with answering the question.
    3. Be concise in answering the question and only answer the question at hand rather than adding extra information.
    </rules>
    Only output the generated answer as a sentence. No extra characters.
    """

In [71]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": generation_template.format(
                context, questions_json_object["questions"][0]
            ),
        }
    ],
    model="gpt-4o-mini",
)

In [72]:
chat_completion.choices[0].message.content

'Henri Sauval a travaillé sur les antiquités de Paris.'